In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def get_molecular_embeddings(smiles_list):
    """
    Calculate molecular embeddings for a list of SMILES strings using ChemBERTa
    
    Parameters:
    smiles_list (list): List of SMILES strings
    
    Returns:
    numpy.ndarray: Normalized embeddings matrix
    """
    # Load ChemBERTa model and tokenizer
    model_name = "seyonec/ChemBERTa-zinc-base-v1"  # Using ChemBERTa instead of MolBERT
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)    
    model.eval()
    
    # Initialize list to store embeddings
    embeddings_list = []
    
    # Process each SMILES string
    with torch.no_grad():
        for smiles in smiles_list:
            try:
                # Tokenize SMILES
                inputs = tokenizer(smiles, return_tensors="pt", padding=True, truncation=True, max_length=512)
                
                # Get model outputs
                outputs = model(**inputs)
                
                # Use CLS token embedding (first token)
                embedding = outputs.last_hidden_state[:, 0, :].numpy()
                embeddings_list.append(embedding.flatten())
                
            except Exception as e:
                print(f"Error processing SMILES: {smiles}")
                print(f"Error message: {str(e)}")
                embeddings_list.append(np.zeros(768))  # ChemBERTa base model has 768 dimensions
    
    embeddings_array = np.array(embeddings_list)
    
    scaler = MinMaxScaler()
    normalized_embeddings = scaler.fit_transform(embeddings_array)
    
    return normalized_embeddings

def process_drug_file(input_file, output_file):
    """
    Process a CSV file containing drug SMILES and save embeddings
    
    Parameters:
    input_file (str): Path to input CSV file with SMILES column
    output_file (str): Path to save output embeddings
    
    Returns:
    pandas.DataFrame: Processed data with embeddings
    """
    # Read the CSV file
    print("Reading CSV file...")
    df = pd.read_csv(input_file)
    
    print(f"Processing {len(df)} SMILES strings...")
    # Calculate embeddings
    embeddings = get_molecular_embeddings(df['SMILES'].tolist())
    
    print("Creating embedding DataFrame...")
    # Create DataFrame with embeddings
    embedding_df = pd.DataFrame(
        embeddings,
        columns=[f'embedding_{i}' for i in range(embeddings.shape[1])]
    )
    
    # Combine original data with embeddings
    result_df = pd.concat([df, embedding_df], axis=1)
    
    print(f"Saving results to {output_file}...")
    # Save to CSV
    result_df.to_csv(output_file, index=False)
    
    return result_df

if __name__ == "__main__":
    input_file = r"Dataset_2016.csv"
    output_file = r"Drugs_With_Embedded.csv"
    
    try:
        result = process_drug_file(input_file, output_file)
        print("Processing completed successfully!")
        print(f"Shape of result: {result.shape}")
    except FileNotFoundError:
        print("Error: Input file not found. Please check the file path.")
    except KeyError:
        print("Error: 'SMILES' column not found in the CSV file.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def normalize_embeddings(embeddings_array):
    """
    Normalize embeddings to range [0,1] using Min-Max scaling
    
    Parameters:
    embeddings_array (numpy.ndarray): Array of embeddings to normalize
    
    Returns:
    numpy.ndarray: Normalized embeddings matrix
    """
    scaler = MinMaxScaler()
    normalized_embeddings = scaler.fit_transform(embeddings_array)
    
    # Verify normalization
    print("\nVerifying normalization:")
    print(f"Minimum value: {np.min(normalized_embeddings)}")
    print(f"Maximum value: {np.max(normalized_embeddings)}")
    
    return normalized_embeddings

def get_molecular_embeddings(smiles_list):
    """
    Calculate molecular embeddings for a list of SMILES strings using ChemBERTa
    
    Parameters:
    smiles_list (list): List of SMILES strings
    
    Returns:
    numpy.ndarray: Normalized embeddings matrix
    """
    # Load ChemBERTa model and tokenizer
    print("Loading ChemBERTa model and tokenizer...")
    model_name = "seyonec/ChemBERTa-zinc-base-v1"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    
    # Set model to evaluation mode
    model.eval()
    
    # Initialize list to store embeddings
    embeddings_list = []
    
    # Process each SMILES string
    print("\nGenerating embeddings...")
    with torch.no_grad():
        for i, smiles in enumerate(smiles_list):
            try:
                # Tokenize SMILES
                inputs = tokenizer(smiles, return_tensors="pt", padding=True, truncation=True, max_length=512)
                
                # Get model outputs
                outputs = model(**inputs)
                
                # Use CLS token embedding (first token)
                embedding = outputs.last_hidden_state[:, 0, :].numpy()
                embeddings_list.append(embedding.flatten())
                
                # Print progress every 100 molecules
                if (i + 1) % 100 == 0:
                    print(f"Processed {i + 1}/{len(smiles_list)} molecules...")
                
            except Exception as e:
                print(f"Error processing SMILES: {smiles}")
                print(f"Error message: {str(e)}")
                # Add a zero vector as embedding for failed cases
                embeddings_list.append(np.zeros(768))  # ChemBERTa base model has 768 dimensions
    
    # Convert list to numpy array
    print("\nConverting to numpy array...")
    embeddings_array = np.array(embeddings_list)
    
    # Print shape and stats before normalization
    print("\nEmbeddings before normalization:")
    print(f"Shape: {embeddings_array.shape}")
    print(f"Min value: {np.min(embeddings_array)}")
    print(f"Max value: {np.max(embeddings_array)}")
    print(f"Mean value: {np.mean(embeddings_array)}")
    
    # Normalize embeddings
    print("\nNormalizing embeddings...")
    normalized_embeddings = normalize_embeddings(embeddings_array)
    
    return normalized_embeddings

def process_drug_file(input_file, output_file):
    """
    Process a CSV file containing drug SMILES and save embeddings
    
    Parameters:
    input_file (str): Path to input CSV file with SMILES column
    output_file (str): Path to save output embeddings
    
    Returns:
    pandas.DataFrame: Processed data with embeddings
    """
    # Read the CSV file
    print("Reading CSV file...")
    df = pd.read_csv(input_file)
    
    print(f"\nFound {len(df)} SMILES strings to process...")
    
    # Check if SMILES column exists
    if 'SMILES' not in df.columns:
        raise KeyError("CSV file must contain a 'SMILES' column")
    
    # Calculate embeddings
    embeddings = get_molecular_embeddings(df['SMILES'].tolist())
    
    print("\nCreating embedding DataFrame...")
    # Create DataFrame with embeddings
    embedding_df = pd.DataFrame(
        embeddings,
        columns=[f'embedding_{i}' for i in range(embeddings.shape[1])]
    )
    
    # Combine original data with embeddings
    result_df = pd.concat([df, embedding_df], axis=1)
    
    print(f"\nSaving results to {output_file}...")
    # Save to CSV
    result_df.to_csv(output_file, index=False)
    
    # Print final statistics
    print("\nFinal statistics:")
    print(f"Original dataframe shape: {df.shape}")
    print(f"Embedding dataframe shape: {embedding_df.shape}")
    print(f"Final dataframe shape: {result_df.shape}")
    
    return result_df

if __name__ == "__main__":
    input_file = r"r"Dataset_2016.csv""
    output_file = r"Drugs_With_Embedded_Bits.csv"
    
    try:
        result = process_drug_file(input_file, output_file)
        print("\nProcessing completed successfully!")
    except FileNotFoundError:
        print("Error: Input file not found. Please check the file path.")
    except KeyError as e:
        print(f"Error: {str(e)}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [ ]:
import pandas as pd
import numpy as np
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score
from scipy.spatial.distance import cdist
from datetime import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer, PowerTransformer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
import hdbscan
from sklearn.neighbors import kneighbors_graph
from scipy.sparse.csgraph import connected_components

def optimize_clustering(X, algorithm):
    """Optimize clustering hyperparameters using Optuna."""
    def objective(trial):
        scaler_type = trial.suggest_categorical('scaler', ['standard', 'minmax', 'robust', 'quantile', 'power'])
        scaler = {'standard': StandardScaler(), 'minmax': MinMaxScaler(), 'robust': RobustScaler(),
                  'quantile': QuantileTransformer(), 'power': PowerTransformer()}[scaler_type]

        X_scaled = scaler.fit_transform(X)

        min_pca = min(5, X_scaled.shape[1])  
        max_pca = min(30, X_scaled.shape[1])  
        if min_pca > max_pca:
            min_pca = max_pca  

        n_components = trial.suggest_int('n_components', min_pca, max_pca)
        X_pca = PCA(n_components=n_components).fit_transform(X_scaled)

        if algorithm == 'kmeans':
            n_clusters = trial.suggest_int('n_clusters', 2, 10)
            model = KMeans(n_clusters=n_clusters, random_state=42)
        elif algorithm == 'hdbscan':
            min_cluster_size = trial.suggest_int('min_cluster_size', 5, 50)
            model = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size)
        elif algorithm == 'spectral':
            n_clusters = trial.suggest_int('n_clusters', 2, 10)
            graph = kneighbors_graph(X_pca, n_neighbors=15, include_self=False)
            n_components_graph, _ = connected_components(graph)
            if n_components_graph > 1:
                return float('-inf')
            model = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', random_state=42)
        elif algorithm == 'agglomerative':
            n_clusters = trial.suggest_int('n_clusters', 2, 10)
            model = AgglomerativeClustering(n_clusters=n_clusters)
        elif algorithm == 'gmm':
            n_components = trial.suggest_int('n_components', 2, 10)
            model = GaussianMixture(n_components=n_components, random_state=42)
        else:
            return float('-inf')

        labels = model.fit_predict(X_pca)
        if len(set(labels)) < 2:
            return float('-inf')

        return silhouette_score(X_pca, labels)

    study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=15, timeout=120)
    return study.best_params

def optimize_linkage(co_assoc_matrix):
    """Optimize linkage method and number of clusters using Optuna."""
    def objective(trial):
        linkage_method = trial.suggest_categorical('linkage_method', ['single', 'complete', 'average', 'ward'])
        linkage_matrix = linkage(1 - co_assoc_matrix, method=linkage_method)

        max_clusters = 10
        t = trial.suggest_int('n_clusters', 2, max_clusters)
        labels = fcluster(linkage_matrix, t, criterion='maxclust')

        if len(set(labels)) < 2:
            return float('-inf')

        return silhouette_score(1 - co_assoc_matrix, labels)

    study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=15)
    return study.best_params

def dunn_index(X, labels):
    unique_clusters = np.unique(labels)
    n_clusters = len(unique_clusters)
    if n_clusters < 2:
        return -1  

    intra_dists = []
    inter_dists = []

    for i in unique_clusters:
        cluster_i = X[labels == i]
        if cluster_i.shape[0] > 1:
            intra_dists.append(np.max(cdist(cluster_i, cluster_i)))
        for j in unique_clusters:
            if i >= j:
                continue
            cluster_j = X[labels == j]
            inter_dists.append(np.min(cdist(cluster_i, cluster_j)))

    if not intra_dists or not inter_dists:
        return -1
    return np.min(inter_dists) / np.max(intra_dists)


def perform_ensemble_clustering(csv_path):
    """Runs multiple clustering algorithms on drug embeddings."""
    print("Loading data...")
    data = pd.read_csv(csv_path)
    
    embedding_cols = [col for col in data.columns if col.startswith('embedding_')]
    if not embedding_cols:
        raise ValueError("No embedding columns found in the dataset.")

    print(f"Found {len(embedding_cols)} embedding dimensions")
    X = data[embedding_cols].values

    print("Starting ensemble clustering...")
    algorithms = ['kmeans', 'hdbscan', 'spectral', 'agglomerative', 'gmm']
    cluster_labels_list = []

    for algo in algorithms:
        print(f"\nOptimizing {algo} clustering...")
        best_params = optimize_clustering(X, algo)
        if best_params is None:
            continue
        print(f"Best parameters for {algo}: {best_params}")

        if algo == 'kmeans':
            model = KMeans(n_clusters=best_params['n_clusters'], random_state=42)
        elif algo == 'hdbscan':
            model = hdbscan.HDBSCAN(min_cluster_size=best_params['min_cluster_size'])
        elif algo == 'spectral':
            model = SpectralClustering(n_clusters=best_params['n_clusters'], 
                                       affinity='nearest_neighbors', random_state=42)
        elif algo == 'agglomerative':
            model = AgglomerativeClustering(n_clusters=best_params['n_clusters'])
        elif algo == 'gmm':
            model = GaussianMixture(n_components=best_params['n_components'], random_state=42)

        print(f"Fitting {algo} model...")
        cluster_labels_list.append(model.fit_predict(X))

    print("\nCreating consensus clustering...")
    n_samples = X.shape[0]
    co_assoc_matrix = np.zeros((n_samples, n_samples))
    for labels in cluster_labels_list:
        for i in range(n_samples):
            for j in range(n_samples):
                if labels[i] == labels[j]:
                    co_assoc_matrix[i, j] += 1
    co_assoc_matrix /= len(algorithms)

    print("\nOptimizing linkage...")
    best_linkage_params = optimize_linkage(co_assoc_matrix)
    print(f"Best linkage parameters: {best_linkage_params}")

    linkage_matrix = linkage(1 - co_assoc_matrix, method=best_linkage_params['linkage_method'])
    final_labels = fcluster(linkage_matrix, t=best_linkage_params['n_clusters'], criterion='maxclust')

    print("\nEvaluating Clustering Performance:")
    silhouette = silhouette_score(1 - co_assoc_matrix, final_labels)
    davies_bouldin = davies_bouldin_score(1 - co_assoc_matrix, final_labels)
    calinski_harabasz = calinski_harabasz_score(1 - co_assoc_matrix, final_labels)
    dunn = dunn_index(1 - co_assoc_matrix, final_labels)

    print(f"Silhouette Score      : {silhouette:.4f}")
    print(f"Davies-Bouldin Index  : {davies_bouldin:.4f}")
    print(f"Calinski-Harabasz     : {calinski_harabasz:.4f}")
    print(f"Dunn Index            : {dunn:.4f}")

    data['Consensus_Cluster'] = final_labels
    
    cluster_stats = pd.DataFrame({
        'Cluster': range(1, best_linkage_params['n_clusters'] + 1),
        'Size': [sum(final_labels == i) for i in range(1, best_linkage_params['n_clusters'] + 1)]
    })
    print("\nCluster Statistics:")
    print(cluster_stats)

    today = datetime.now().strftime('%Y-%m-%d')
    output_path = csv_path.replace('.csv', f'_clustered_{today}.csv')
    data.to_csv(output_path, index=False)
    print(f"\nClustered data saved to: {output_path}")

    return final_labels

if __name__ == "__main__":
    csv_path = r"Drugs_With_Embedded_Bits.csv"
    final_clusters = perform_ensemble_clustering(csv_path)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

csv_path = r"Drugs_With_Embedded_clustered_2025-02-13.csv"
df = pd.read_csv(csv_path)

if 'Consensus_Cluster' not in df.columns:
    raise ValueError("CSV file must contain a column named 'Consensus_Cluster'.")

numerical_data = df.select_dtypes(include=[np.number])

imputer = SimpleImputer(strategy="mean")
X_imputed = imputer.fit_transform(numerical_data)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

cluster_labels = df['Consensus_Cluster']

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=cluster_labels, palette="tab10", s=80, edgecolor="black")
plt.title("PCA Projection of Clusters")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.legend(title="Cluster")
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 6))
sns.heatmap(pd.crosstab(df['Consensus_Cluster'], df['Consensus_Cluster']), annot=True, cmap="coolwarm", fmt="d")
plt.title("Cluster Distribution Heatmap")
plt.xlabel("Cluster ID")
plt.ylabel("Cluster ID")
plt.show()

num_features = numerical_data.columns[:6]
df_imputed = pd.DataFrame(X_imputed, columns=numerical_data.columns)
df_imputed['Consensus_Cluster'] = cluster_labels

plt.figure(figsize=(12, 8))
for i, feature in enumerate(num_features):
    plt.subplot(2, 3, i + 1)
    sns.boxplot(x='Consensus_Cluster', y=feature, data=df_imputed, palette="Set2")
    plt.title(f"Boxplot of {feature} by Cluster")
    plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

sns.pairplot(df_imputed[num_features.tolist() + ['Consensus_Cluster']], hue="Consensus_Cluster", palette="tab10", diag_kind="kde")
plt.suptitle("Pairplot of Clusters", y=1.02)
plt.show()

plt.figure(figsize=(8, 5))
sns.countplot(x=cluster_labels, palette="viridis")
plt.title("Cluster Size Distribution")
plt.xlabel("Cluster ID")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.show()

print("All visualizations generated successfully!")
